In [1]:
!pip install torch torchtext transformers sentencepiece pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.9 MB/s eta 0:00:00


In [2]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [4]:
#load data
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [6]:
updated_data = [{'Name': item['Name'], 'Symptoms': item['Symptoms']} for item in data_sample['train']]

In [7]:
df = pd.DataFrame(updated_data)

In [8]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [9]:
#just extract symptoms
df['Symptoms'] = df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [11]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')


In [12]:
device

device(type='cuda')

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [15]:
BATCH_SIZE = 8

In [16]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [17]:
# Dataset Prep
class LanguageDataset(Dataset):
  def __init__(self, df, tokenizer):
    self.labels = df.columns
    self.data = df.to_dict(orient='records')
    self.tokenizer = tokenizer
    x = self.fittest_max_length(df)
    self.max_length = x

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    x = self.data[idx][self.labels[0]]
    y = self.data[idx][self.labels[1]]
    text = f"{x} | {y}"
    tokens = self.tokenizer.encode_plus(text, return_tensors= 'pt', max_length=128, padding='max_length', truncation = True)
    return tokens

  def fittest_max_length(self, df):
    max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
    x = 2
    while x < max_length: x = x * 2
    return x

In [18]:
data_sample = LanguageDataset(df, tokenizer)

In [19]:
data_sample

In [20]:
train_size = int(0.8 * len(data_sample))
val_size = len(data_sample) - train_size

train_data, val_data = random_split(data_sample, [train_size, val_size])

In [21]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [22]:
num_epochs = 8

In [23]:
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [24]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [25]:
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu', 'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [44]:
#training loop
for epoch in range(num_epochs):
  start_time = time.time()
  model.train()
  epoch_training_loss = 0
  train_iterator = tqdm(train_loader, desc=f"Training epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
  for batch in train_iterator:
    optimizer.zero_grad()
    inputs = batch['input_ids'].squeeze(1).to(device)
    targets = inputs.clone()
    outputs = model(input_ids= inputs, labels=targets)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    train_iterator.set_postfix({'Training Loss': loss.item()})
    epoch_training_loss += loss.item()
  avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

  #validation
  model.eval()
  epoch_validation_loss = 0
  total_loss = 0
  valid_iterator = tqdm(val_loader, desc=f'validation epoch {epoch+1}/{num_epochs}')
  with torch.no_grad():
    for batch in valid_iterator:
      inputs = batch['input_ids'].squeeze(1).to(device)
      targets = inputs.clone()
      outputs = model(input_ids= inputs, labels=targets)
      loss = outputs.loss
      total_loss += loss
      valid_iterator.set_postfix({'validation Loss': loss.item()})
      epoch_validation_loss += loss.item()
  avg_epoch_validation_loss = epoch_validation_loss / len(valid_iterator)

  end_time = time.time()
  epoch_duration_sec = end_time - start_time

  new_row = {
      'transformer': model_name,
      'batch_size': batch_size,
      'gpu': gpu,
      'epoch': epoch+1,
      'training_loss': avg_epoch_training_loss,
      'validation_loss': avg_epoch_validation_loss,
      'epoch_duration_sec': epoch_duration_sec
  }
  results.loc[len(results)] = new_row
  print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")



Training epoch 1/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.62it/s, Training Loss=0.0879]
validation epoch 1/8: 100%|██████████| 10/10 [00:00<00:00, 15.74it/s, validation Loss=0.827]


Epoch: 1, Validation Loss: 0.9540582895278931


Training epoch 2/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.53it/s, Training Loss=0.059]
validation epoch 2/8: 100%|██████████| 10/10 [00:00<00:00, 15.75it/s, validation Loss=0.78]


Epoch: 2, Validation Loss: 0.9082280993461609


Training epoch 3/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.51it/s, Training Loss=0.0486]
validation epoch 3/8: 100%|██████████| 10/10 [00:00<00:00, 15.65it/s, validation Loss=0.841]


...

Epoch: 6, Validation Loss: 1.0043903589248657


Training epoch 7/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.87it/s, Training Loss=0.0457]
validation epoch 7/8: 100%|██████████| 10/10 [00:00<00:00, 16.24it/s, validation Loss=0.889]


Epoch: 7, Validation Loss: 1.0190497636795044


Training epoch 8/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.86it/s, Training Loss=0.0638]
validation epoch 8/8: 100%|██████████| 10/10 [00:00<00:00, 16.80it/s, validation Loss=0.927]

Epoch: 8, Validation Loss: 1.0541629791259766


In [37]:
input_str = 'Pneumonia'

In [38]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

In [39]:
input_ids

tensor([[   47, 25668, 11339]], device='cuda:0')

In [40]:
output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences = 1,
    do_sample= True,
    top_k = 8,
    top_p = 0.95,
    temperature = 0.5,
    repetition_penalty=1.2
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [41]:
output

tensor([[   47, 25668, 11339,   930,   399,   682,    64,    11, 30874,    11,
         32692,  2356,   393,  1067,  9430,   357,   259,   617,  2663,     8]],
       device='cuda:0')

In [42]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens = True)

In [43]:
decoded_output

'Pneumonia | Nausea, vomiting, abdominal pain or cramps (in some cases)'